# Lab 2B: Direct APIM Access (No Foundry Spoke)

Demonstrate how teams can access the Landing Zone models **directly via APIM** without a Foundry spoke.

## Use Case: Northwind Legacy Integration

Some teams may:
- Have existing applications that can't use the Agent Service
- Need simple REST API access to models
- Be migrating from other platforms
- Want lightweight integration without Foundry overhead

## Step 1: Load APIM Configuration

In [ ]:
import os

# Load the .env file
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

APIM_URL = os.environ['APIM_URL']
APIM_KEY = os.environ['APIM_KEY']

print(f"APIM URL: {APIM_URL}")
print(f"APIM Key: {APIM_KEY[:8]}... (hidden)")

## Step 2: Create a Dedicated APIM Subscription for Northwind

For production, each team should have their own APIM subscription for tracking and access control.

In [63]:
import subprocess
import json

RG = "foundry-lz-parent"
APIM_NAME = APIM_URL.split('//')[1].split('.')[0]  # Extract APIM name from URL
SUB_ID = subprocess.run('az account show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()

# Create a subscription for Northwind
NORTHWIND_SUB_NAME = "northwind-legacy-access"

sub_body = {
    "properties": {
        "displayName": "Northwind Legacy Integration Access",
        "scope": f"/apis/openai",
        "state": "active"
    }
}

# Create via REST API
uri = f"https://management.azure.com/subscriptions/{SUB_ID}/resourceGroups/{RG}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/subscriptions/{NORTHWIND_SUB_NAME}?api-version=2024-06-01-preview"

import tempfile
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(sub_body, f)
    body_file = f.name

result = subprocess.run(
    f'az rest --method PUT --uri "{uri}" --body @{body_file} -o json',
    shell=True, capture_output=True, text=True
)

os.unlink(body_file)

if result.returncode == 0:
    print(f"✅ Created APIM subscription: {NORTHWIND_SUB_NAME}")
else:
    print(f"ℹ️ Subscription may already exist: {result.stderr[:100]}")

✅ Created APIM subscription: northwind-legacy-access


In [64]:
# Get the subscription key
key_uri = f"https://management.azure.com/subscriptions/{SUB_ID}/resourceGroups/{RG}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/subscriptions/{NORTHWIND_SUB_NAME}/listSecrets?api-version=2024-06-01-preview"

result = subprocess.run(
    f'az rest --method POST --uri "{key_uri}" --query primaryKey -o tsv',
    shell=True, capture_output=True, text=True
)

NORTHWIND_KEY = result.stdout.strip()
print(f"Northwind APIM Key: {NORTHWIND_KEY[:8]}... (hidden)")

Northwind APIM Key: 46a94357... (hidden)


## Step 3: Use AzureOpenAI SDK with APIM

Use `AzureOpenAI` client with `api_key` authentication pointing to APIM.

In [65]:
!pip install openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from openai import AzureOpenAI

# AzureOpenAI with APIM - uses api_key auth (not Entra)
client = AzureOpenAI(
    azure_endpoint=APIM_URL.replace("/openai", ""),  # Base without /openai suffix
    api_key=NORTHWIND_KEY,
    api_version="2024-10-21"
)

print("✅ AzureOpenAI client configured for APIM")
print(f"Endpoint: {APIM_URL.replace('/openai', '')}")

## Step 4: Test Direct Chat Completions

Call models directly without Agent Service!

In [ ]:
# Test with gpt-4.1-mini
print("Testing gpt-4.1-mini via direct APIM...")

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a space exploration expert for the Galactic Discovery Agency."},
        {"role": "user", "content": "What are the best planets to visit for a first-time space tourist?"}
    ],
    max_tokens=150
)

print(f"\n✅ Model: {response.model}")
print(f"Response: {response.choices[0].message.content}")

Testing gpt-4.1-mini via direct APIM...

✅ Model: gpt-4.1-mini-2025-04-14
Response: For a bakery, I recommend the following products to ensure you have a well-stocked inventory for baking and selling your goods:

1. **Flour and Baking Mixes**  
   - All-purpose flour  
   - Bread flour  
   - Cake flour  
   - Self-rising flour  
   - Baking powder and baking soda

2. **Sweeteners**  
   - Granulated sugar  
   - Brown sugar  
   - Powdered sugar  
   - Honey  
   - Molasses

3. **Dairy and Fats**  
   - Butter  
   - Margarine  
   - Shortening  
   - Milk and cream  
   - Eggs

4. **Flavorings and Spices**  
   - Vanilla extract  
  


In [ ]:
# Test with different prompts
prompts = [
    "Name the closest star to Earth.",
    "How many moons does Jupiter have?",
    "What causes a solar eclipse?"
]

print("Testing gpt-4.1-mini with space-themed prompts:")
print("="*50)

for prompt in prompts:
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=30
    )
    print(f"Q: {prompt}")
    print(f"A: {response.choices[0].message.content}\n")

Testing gpt-4.1-mini with multiple prompts:
Q: Say hello in one sentence.
A: Hello! How can I assist you today?

Q: What is 2+2?
A: 2 + 2 equals 4.

Q: Name one planet.
A: Earth



## Step 5: Streaming Example

Direct APIM access also supports streaming!

In [ ]:
print("Streaming response from gpt-4.1-mini:")
print("-" * 40)

stream = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": "Write a haiku about exploring Mars."}],
    stream=True
)

for chunk in stream:
    if chunk.choices and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n" + "-" * 40)
print("✅ Streaming complete!")

Streaming response from gpt-4.1-mini:
----------------------------------------
Silent clouds above,  
Data flows through unseen streams,  
Dreams built in the sky.
----------------------------------------
✅ Streaming complete!


## Step 6: Using with LangChain (Optional)

LangChain also works with direct APIM access!

In [70]:
!pip install langchain-openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=APIM_URL.replace("/openai", ""),
    api_key=NORTHWIND_KEY,
    api_version="2024-10-21",
    azure_deployment="gpt-4.1-mini"
)

response = llm.invoke("What is the Great Red Spot on Jupiter?")
print(f"LangChain via APIM: {response.content}")

LangChain via APIM: The capital of France is Paris.


## Summary: Direct APIM vs Agent Service

### Configuration Pattern

```python
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint="https://your-apim.azure-api.net",  # No /openai suffix
    api_key="your-apim-subscription-key",
    api_version="2024-10-21"
)
```

### When to Use

| Direct APIM | Agent Service |
|-------------|---------------|
| ✅ Migrating existing apps | ✅ New AI applications |
| ✅ Simple chat completions | ✅ Agent memory/state |
| ✅ Third-party frameworks | ✅ Tools & function calling |
| ✅ Streaming support | ✅ Multi-agent workflows |

**Next**: Lab 2C - Multi-Model Agents

## Save Northwind Configuration

In [72]:
from pathlib import Path

# Append to .env
env_file = Path('/workspaces/getting-started-with-foundry/.env')
with open(env_file, 'a') as f:
    f.write(f"\n# Northwind Direct APIM Access\n")
    f.write(f"NORTHWIND_APIM_KEY={NORTHWIND_KEY}\n")

print(f"✅ Northwind configuration saved to .env")

✅ Northwind configuration saved to .env
